# Claycode SIGGRAPH Evaluation

In [18]:
import pandas as pd
import plotly.express as px

# Load the dataset
df = pd.read_csv('results.csv')

# Process the 'type' column to categorize types based on the filename
df['type'] = df['filename'].str.split('-').map(lambda x: "-".join(x[:-1]).replace("_clay", "clay"))

# Select relevant columns
df = df[['successful', 'square_dimension_perc', 'type']]

# Group by square_dimension_perc and type, then calculate success rate
grouped = df.groupby(['square_dimension_perc', 'type'])
df = grouped['successful'].agg(['sum', 'count']).reset_index()
df['success_rate'] = df['sum'] / df['count']
df['type'] = df['type'].replace({'claycode': 'Claycode (R=1)',
                                 'claycode-fragment': 'Claycode (R=2)',
                                 'qr': 'QR',
                                 'qrart': 'Styl. QR',
                                 'claycodeart': 'Styl. Claycode (R=2)',
                                 'barcode': 'Barcode'
                                 })

# Define the custom color scheme using Dark2[0], Dark2[1], Dark2[2]
color_scheme = {
    'Claycode (R=1)': '#1b9e77', 
    'Styl. Claycode (R=2)': '#1b9e77', 
    'Claycode (R=2)': '#1b9e77', 
    'QR': '#d95f02', 
    'Styl. QR': '#d95f02', 
    'Barcode': '#7570b3'
}

# Define line dash style
line_dash_map = {
    'Claycode (R=1)': 'solid', 
    'Styl. Claycode (R=2)': 'dot',  
    'Claycode (R=2)': 'dashdot',  
    'QR': 'solid', 
    'Styl. QR': 'dot', 
    'Barcode': 'solid'
}

# Define line width and marker symbols
line_width_map = {
    'Claycode (R=1)': 2,
    'Styl. Claycode (R=2)': 2, 
    'Claycode (R=2)': 2, 
    'QR': 2, 
    'Styl. QR': 2, 
    'Barcode': 2
}

marker_map = {
    'QR': 'square-open',
    'Styl. QR': 'square',
    'Claycode (R=1)': 'hexagram-open',
    'Styl. Claycode (R=2)': 'hexagram-dot',
    'Claycode (R=2)': 'star',
    'Barcode': 'cross',
}

# Apply small offsets to the success rate for better separation between overlapping claycode types
df['adjusted_success_rate'] = df['success_rate']
df.loc[df['type'] == 'Styl. Claycode (R=2)', 'adjusted_success_rate'] += 0.01
df.loc[df['type'] == 'claycode (R=2)', 'adjusted_success_rate'] += 0.02

# Create the line plot with custom color, line styles, and width
fig = px.line(
    df,
    x="square_dimension_perc",
    y="adjusted_success_rate",
    color="type",
    line_dash="type",
    markers=True,
    labels={
        "square_dimension_perc": "Square side dimension (%)",
        "adjusted_success_rate": "Success rate (%)",
        "type": "Type"
    },
    color_discrete_map=color_scheme,
)

# Apply line width, dash style, and marker symbols
for trace in fig.data:
    trace.line.dash = line_dash_map[trace.name]
    trace.line.width = line_width_map[trace.name]
    trace.marker.symbol = marker_map[trace.name]
    trace.marker.size = 13
    trace.marker.opacity = 1

# Move the 'Barcode' trace to the foreground by reordering
fig.data = sorted(fig.data, key=lambda x: x.name != 'Barcode', reverse=True)

# Update layout
fig.update_layout(
    height=400,
    width=700,
    margin=dict(t=30, b=50, l=50, r=50),
    legend=dict(
        x=0.7,
        y=0.9,
        xanchor='left',
        yanchor='top',
        bgcolor='rgba(255, 255, 255, 0.7)',
        bordercolor='black',
        borderwidth=1,
        title=""
    )
)

# Update axes
fig.update_xaxes(range=[0.09, 0.52], tickformat='.0%')
fig.update_yaxes(range=[-0.05, 1.05], tickformat='.0%')

# Export to PDF
fig.write_image("square-no-sinusoidal.pdf", format="pdf", engine="kaleido")
fig.show()